<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStardust/blob/main/Complete_MAML_Example_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

# Define a simple neural network
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Function to generate synthetic data for tasks
def generate_task_data(num_tasks=10, num_samples=20):
    tasks = []
    for _ in range(num_tasks):
        # Random linear function parameters
        slope = np.random.rand()
        intercept = np.random.rand()

        # Generate samples
        x = np.random.rand(num_samples, 1).astype(np.float32) * 10  # Random x values
        y = slope * x + intercept + np.random.normal(0, 0.5, size=x.shape).astype(np.float32)  # y = mx + b + noise

        tasks.append((torch.tensor(x), torch.tensor(y)))
    return tasks

# MAML training function
def train_maml(model, tasks, inner_steps=1, outer_steps=10, inner_lr=0.01, outer_lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=outer_lr)

    for outer_step in range(outer_steps):
        meta_loss = 0

        for task in tasks:
            x_train, y_train = task

            # Clone the model for inner updates
            temp_model = NeuralNet()
            temp_model.load_state_dict(model.state_dict())

            # Inner loop (fine-tuning)
            inner_optimizer = optim.SGD(temp_model.parameters(), lr=inner_lr)
            for _ in range(inner_steps):
                y_pred_train = temp_model(x_train)
                inner_loss = nn.MSELoss()(y_pred_train.squeeze(), y_train.squeeze())
                inner_optimizer.zero_grad()
                inner_loss.backward()
                inner_optimizer.step()

            # Outer loop (meta-update)
            x_test, y_test = generate_task_data(num_tasks=1)[0]  # Generate new test data for this task
            y_pred_test = temp_model(x_test)
            outer_loss = nn.MSELoss()(y_pred_test.squeeze(), y_test.squeeze())
            meta_loss += outer_loss

        # Average meta loss and update model parameters
        meta_loss /= len(tasks)
        optimizer.zero_grad()
        meta_loss.backward()
        optimizer.step()

        print(f'Outer Step {outer_step + 1}/{outer_steps}, Meta Loss: {meta_loss.item():.4f}')

# Main execution
if __name__ == "__main__":
    num_tasks = 5  # Number of tasks to generate
    tasks = generate_task_data(num_tasks=num_tasks)

    model_maml = NeuralNet()
    train_maml(model_maml, tasks)